In [1]:
import numpy as np
import pandas as pd
#https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction
#https://www.kaggle.com/datasets/alexisbcook/synthetic-credit-card-approval
import os
import kagglehub
import warnings

import os
warnings.filterwarnings("ignore")

df = pd.read_csv("/opt/notebooks/Chatbot-Credit-Card/backend/dataset/credit-card-approval/credit_card_approval.csv")
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,JOB,BEGIN_MONTHS,STATUS,TARGET
0,5065438,F,Y,N,2+ children,270000.0,Secondary / secondary special,Married,With parents,-13258,-2300,1,0,0,0,Managers,-6,C,0
1,5142753,F,N,N,No children,81000.0,Secondary / secondary special,Single / not married,House / apartment,-17876,-377,1,1,1,0,Private service staff,-4,0,0
2,5111146,M,Y,Y,No children,270000.0,Higher education,Married,House / apartment,-19579,-1028,1,0,1,0,Laborers,0,C,0
3,5010310,F,Y,Y,1 children,112500.0,Secondary / secondary special,Married,House / apartment,-15109,-1956,1,0,0,0,Core staff,-3,0,0
4,5010835,M,Y,Y,2+ children,139500.0,Secondary / secondary special,Married,House / apartment,-17281,-5578,1,1,0,0,Drivers,-29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537662,5142999,M,Y,N,1 children,166500.0,Secondary / secondary special,Married,With parents,-12372,-5401,1,0,1,0,Core staff,-8,0,0
537663,5010773,F,N,Y,No children,135000.0,Higher education,Married,With parents,-14160,-4635,1,0,0,0,Sales staff,-8,0,0
537664,5105601,M,N,Y,No children,180000.0,Higher education,Married,House / apartment,-24204,-2462,1,0,0,0,Private service staff,-7,0,0
537665,5132833,M,Y,N,No children,220500.0,Secondary / secondary special,Married,House / apartment,-22647,-3847,1,0,1,0,Laborers,-1,0,0


In [2]:
df.rename(columns={'TARGET': 'Approved'}, inplace=True)
# Display the first few rows to confirm the change
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,JOB,BEGIN_MONTHS,STATUS,Approved
0,5065438,F,Y,N,2+ children,270000.0,Secondary / secondary special,Married,With parents,-13258,-2300,1,0,0,0,Managers,-6,C,0
1,5142753,F,N,N,No children,81000.0,Secondary / secondary special,Single / not married,House / apartment,-17876,-377,1,1,1,0,Private service staff,-4,0,0
2,5111146,M,Y,Y,No children,270000.0,Higher education,Married,House / apartment,-19579,-1028,1,0,1,0,Laborers,0,C,0
3,5010310,F,Y,Y,1 children,112500.0,Secondary / secondary special,Married,House / apartment,-15109,-1956,1,0,0,0,Core staff,-3,0,0
4,5010835,M,Y,Y,2+ children,139500.0,Secondary / secondary special,Married,House / apartment,-17281,-5578,1,1,0,0,Drivers,-29,0,0


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import colorama
from colorama import Fore

# Automatically identify categorical columns from the dataframe based on data types
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Identify potential categorical columns stored as numbers by checking unique values
for col in df.columns:
    if col != 'Approved':  # Skip 'Approved' column
        if df[col].nunique() < 10 and col not in categorical_cols:  # Arbitrary threshold of < 10 unique values
            categorical_cols.append(col)

# Display identified categorical columns
print(f"Identified Categorical Columns (excluding 'Approved'): {categorical_cols}\n")


Identified Categorical Columns (excluding 'Approved'): ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'JOB', 'STATUS', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL']



In [4]:
# Initialize a list to store bad predictors
bad_predictors = []

# You can change the threshold for what is considered a "bad predictor"
p_value_threshold = 0.05  # Adjust this if needed

# Evaluate each categorical column using chi-squared test
for col in categorical_cols:
    print(f"{col}:")
    
    # Perform chi-squared test
    a = np.array(pd.crosstab(df['Approved'], df[col]))
    stats, p, dof, _ = chi2_contingency(a, correction=False)
    
    if p > p_value_threshold:
        print(Fore.RED + f"'{col}' is a 'bad Predictor'")
        print(f"p_val = {p}\n")
        bad_predictors.append(col)  # Append the bad predictor to the list
    else:
        print(Fore.GREEN + f"'{col}' is a 'Good Predictor'")
        print(f"p_val = {p}\n")

CODE_GENDER:
'CODE_GENDER' is a 'Good Predictor'
p_val = 1.3251901990018294e-34

FLAG_OWN_CAR:
'FLAG_OWN_CAR' is a 'Good Predictor'
p_val = 0.024096469399418216

FLAG_OWN_REALTY:


'FLAG_OWN_REALTY' is a 'Good Predictor'
p_val = 8.382674884394374e-05

CNT_CHILDREN:


'CNT_CHILDREN' is a 'Good Predictor'
p_val = 2.911490124240846e-11

NAME_EDUCATION_TYPE:
'NAME_EDUCATION_TYPE' is a 'Good Predictor'
p_val = 5.161166610168622e-19

NAME_FAMILY_STATUS:


'NAME_FAMILY_STATUS' is a 'Good Predictor'
p_val = 1.0607740863942537e-37

NAME_HOUSING_TYPE:


'NAME_HOUSING_TYPE' is a 'Good Predictor'
p_val = 9.253623067245047e-10

JOB:
'JOB' is a 'Good Predictor'
p_val = 7.782512719963732e-132

STATUS:


'STATUS' is a 'Good Predictor'
p_val = 0.0

FLAG_MOBIL:
'FLAG_MOBIL' is a 'bad Predictor'
p_val = 1.0

FLAG_WORK_PHONE:


'FLAG_WORK_PHONE' is a 'Good Predictor'
p_val = 0.0014145250589105126

FLAG_PHONE:
'FLAG_PHONE' is a 'Good Predictor'
p_val = 2.119534116044579e-10

FLAG_EMAIL:
'FLAG_EMAIL' is a 'bad Predictor'
p_val = 0.18517623597764232



In [5]:
# Automatically drop bad predictors
df.drop(bad_predictors, axis=1, inplace=True)

# Print the final dataframe structure after dropping
print(f"Dropped bad predictors: {bad_predictors}")
df

Dropped bad predictors: ['FLAG_MOBIL', 'FLAG_EMAIL']


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,JOB,BEGIN_MONTHS,STATUS,Approved
0,5065438,F,Y,N,2+ children,270000.0,Secondary / secondary special,Married,With parents,-13258,-2300,0,0,Managers,-6,C,0
1,5142753,F,N,N,No children,81000.0,Secondary / secondary special,Single / not married,House / apartment,-17876,-377,1,1,Private service staff,-4,0,0
2,5111146,M,Y,Y,No children,270000.0,Higher education,Married,House / apartment,-19579,-1028,0,1,Laborers,0,C,0
3,5010310,F,Y,Y,1 children,112500.0,Secondary / secondary special,Married,House / apartment,-15109,-1956,0,0,Core staff,-3,0,0
4,5010835,M,Y,Y,2+ children,139500.0,Secondary / secondary special,Married,House / apartment,-17281,-5578,1,0,Drivers,-29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537662,5142999,M,Y,N,1 children,166500.0,Secondary / secondary special,Married,With parents,-12372,-5401,0,1,Core staff,-8,0,0
537663,5010773,F,N,Y,No children,135000.0,Higher education,Married,With parents,-14160,-4635,0,0,Sales staff,-8,0,0
537664,5105601,M,N,Y,No children,180000.0,Higher education,Married,House / apartment,-24204,-2462,0,0,Private service staff,-7,0,0
537665,5132833,M,Y,N,No children,220500.0,Secondary / secondary special,Married,House / apartment,-22647,-3847,0,1,Laborers,-1,0,0


In [6]:
df.columns = df.columns.str.strip()  # Removes leading and trailing spaces from all column names
# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Print columns to verify correctness
print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

# Remove 'Approved' if it's in either list
if 'Approved' in categorical_cols:
    categorical_cols.remove('Approved')
if 'Approved' in numerical_cols:
    numerical_cols.remove('Approved')

# Check the results
print("Categorical columns after removal:", categorical_cols)
print("Numerical columns after removal:", numerical_cols)

Categorical columns: ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'JOB', 'STATUS']
Numerical columns: ['ID', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'BEGIN_MONTHS', 'Approved']
Categorical columns after removal: ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'JOB', 'STATUS']
Numerical columns after removal: ['ID', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'BEGIN_MONTHS']


In [7]:
print("Data types of the columns:")
print(df.dtypes)

print("\nCategorical columns identified:")
print(categorical_cols)

Data types of the columns:
ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN            object
AMT_INCOME_TOTAL       float64
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
JOB                     object
BEGIN_MONTHS             int64
STATUS                  object
Approved                 int64
dtype: object

Categorical columns identified:
['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'JOB', 'STATUS']


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Initialize LabelEncoders for categorical variables
label_encoders = {}
for col in categorical_cols:
    if col != 'Approved':  # Exclude the target column from encoding
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le  # Store the encoder for future use (e.g., inverse_transform)

print("Unique values in categorical columns after encoding:")
for col in categorical_cols:
    if col != 'Approved':  # Ensure target is not being encoded
        print(f"{col}: {df[col].unique()}")
print("\nData types of the columns after encoding:")
print(df.dtypes)


Unique values in categorical columns after encoding:
CODE_GENDER: [0 1]
FLAG_OWN_CAR: [1 0]
FLAG_OWN_REALTY: [0 1]
CNT_CHILDREN: [1 2 0]
NAME_EDUCATION_TYPE: [4 1 2 3 0]
NAME_FAMILY_STATUS: [1 3 0 2 4]
NAME_HOUSING_TYPE: [5 1 4 2 0 3]
JOB: [10 12  8  3  4  6 13 15  0 14 11 17  9  1  5  2 16  7]
STATUS: [6 0 7 2 1 5 3 4]

Data types of the columns after encoding:
ID                       int64
CODE_GENDER              int64
FLAG_OWN_CAR             int64
FLAG_OWN_REALTY          int64
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_EDUCATION_TYPE      int64
NAME_FAMILY_STATUS       int64
NAME_HOUSING_TYPE        int64
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
JOB                      int64
BEGIN_MONTHS             int64
STATUS                   int64
Approved                 int64
dtype: object


In [9]:
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,JOB,BEGIN_MONTHS,STATUS,Approved
0,5065438,0,1,0,1,270000.0,4,1,5,-13258,-2300,0,0,10,-6,6,0
1,5142753,0,0,0,2,81000.0,4,3,1,-17876,-377,1,1,12,-4,0,0
2,5111146,1,1,1,2,270000.0,1,1,1,-19579,-1028,0,1,8,0,6,0
3,5010310,0,1,1,0,112500.0,4,1,1,-15109,-1956,0,0,3,-3,0,0
4,5010835,1,1,1,1,139500.0,4,1,1,-17281,-5578,1,0,4,-29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537662,5142999,1,1,0,0,166500.0,4,1,5,-12372,-5401,0,1,3,-8,0,0
537663,5010773,0,0,1,2,135000.0,1,1,5,-14160,-4635,0,0,14,-8,0,0
537664,5105601,1,0,1,2,180000.0,1,1,1,-24204,-2462,0,0,12,-7,0,0
537665,5132833,1,1,0,2,220500.0,4,1,1,-22647,-3847,0,1,8,-1,0,0


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame and 'Approved' is the target column

# Select numerical features (excluding 'Approved')
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Remove 'Approved' from the list of columns to scale
if 'Approved' in numerical_cols:
    numerical_cols.remove('Approved')

# Scale only the numerical feature columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Separate features (X) and target variable (y)
X = df.drop('Approved', axis=1)  # Drop 'Approved' from features
y = df['Approved']  # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting splits
print("Training set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)


Training set: (430133, 16) (430133,)
Test set: (107534, 16) (107534,)


In [11]:
print(df['Approved'].value_counts())  # Ensure the target has values like 0/1 or yes/no
df

Approved
0    535705
1      1962
Name: count, dtype: int64


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,JOB,BEGIN_MONTHS,STATUS,Approved
0,-0.328397,-0.781404,1.150190,-1.340706,-0.473859,0.699862,0.695138,-0.334471,3.705763,0.513099,0.193002,-0.626108,-0.652929,0.555688,0.947814,0.688706,0
1,1.512351,-0.781404,-0.869422,-1.340706,0.708649,-1.115022,0.695138,1.972159,-0.310019,-0.838610,0.996288,1.597169,1.531560,1.022220,1.090287,-1.260846,0
2,0.759838,1.279748,1.150190,0.745876,0.708649,0.699862,-1.508671,-0.334471,-0.310019,-1.337086,0.724348,-0.626108,1.531560,0.089157,1.375231,0.688706,0
3,-1.640907,-0.781404,1.150190,0.745876,-1.656368,-0.812541,0.695138,-0.334471,-0.310019,-0.028697,0.336699,-0.626108,-0.652929,-1.077172,1.161523,-1.260846,0
4,-1.628408,1.279748,1.150190,0.745876,-0.473859,-0.553272,0.695138,-0.334471,-0.310019,-0.664451,-1.176302,1.597169,-0.652929,-0.843907,-0.690617,-1.260846,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537662,1.518207,1.279748,1.150190,-1.340706,-1.656368,-0.294003,0.695138,-0.334471,3.705763,0.772435,-1.102365,-0.626108,1.531560,-1.077172,0.805342,-1.260846,0
537663,-1.629884,-0.781404,-0.869422,0.745876,0.708649,-0.596484,-1.508671,-0.334471,3.705763,0.249080,-0.782387,-0.626108,-0.652929,1.488751,0.805342,-1.260846,0
537664,0.627820,1.279748,-0.869422,0.745876,0.708649,-0.164368,-1.508671,-0.334471,-0.310019,-2.690844,0.125330,-0.626108,-0.652929,1.022220,0.876578,-1.260846,0
537665,1.276171,1.279748,1.150190,-1.340706,0.708649,0.224536,0.695138,-0.334471,-0.310019,-2.235104,-0.453220,-0.626108,1.531560,0.089157,1.303995,-1.260846,0


In [12]:
X_train

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,JOB,BEGIN_MONTHS,STATUS
124593,0.403474,-0.781404,-0.869422,-1.340706,0.708649,0.699862,-1.508671,1.972159,-0.310019,-0.175635,-0.081862,-0.626108,-0.652929,0.089157,-2.186577,-1.260846
334751,1.507684,-0.781404,-0.869422,0.745876,0.708649,-0.790936,0.695138,3.125475,-0.310019,-1.555444,-0.606107,1.597169,1.531560,1.955283,1.161523,1.013632
466501,0.282289,-0.781404,-0.869422,0.745876,-0.473859,1.564093,-1.508671,-0.334471,-0.310019,1.266228,0.375548,1.597169,-0.652929,0.555688,0.235453,0.688706
474866,0.760981,-0.781404,-0.869422,0.745876,0.708649,0.181324,0.695138,-0.334471,-0.310019,-1.130437,-3.929530,1.597169,1.531560,0.089157,1.303995,0.688706
407920,0.205197,-0.781404,1.150190,-1.340706,0.708649,0.051689,0.695138,-0.334471,-0.310019,-1.342648,-3.833871,-0.626108,1.531560,0.788954,-1.189271,1.013632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,0.869714,1.279748,1.150190,0.745876,0.708649,0.699862,0.695138,-0.334471,-0.310019,0.178538,-0.261901,-0.626108,-0.652929,0.089157,-0.191964,0.688706
259178,0.154962,1.279748,-0.869422,0.745876,0.708649,0.699862,0.695138,0.818844,-0.310019,0.476511,0.739804,1.597169,1.531560,0.089157,1.090287,0.688706
365838,1.344430,-0.781404,-0.869422,-1.340706,0.708649,-0.596484,0.695138,-0.334471,-0.310019,1.172855,0.497941,-0.626108,-0.652929,1.022220,1.303995,0.688706
131932,0.884880,-0.781404,-0.869422,0.745876,0.708649,-0.164368,-1.508671,1.972159,-0.310019,-0.738213,-0.423561,-0.626108,-0.652929,0.788954,-0.191964,1.013632


In [13]:
# Reduce dataset size using stratified sampling (optional)
# X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.1, stratify=y, random_state=42)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, accuracy_score, roc_auc_score, f1_score

# Set up K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# List of models to evaluate with optimizations
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42, ccp_alpha=0.01, max_depth=6, min_samples_split=20),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=50, max_depth=6, n_jobs=-1, min_samples_split=10, max_features='sqrt'),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000, C=0.5, n_jobs=-1),  # Added regularization with C=0.5
    "XGBoost": XGBClassifier(random_state=42, eval_metric='logloss', tree_method='gpu_hist', n_estimators=50, max_depth=6, subsample=0.8, colsample_bytree=0.8, reg_lambda=10)  # Added L2 regularization with reg_lambda=10
}

# Dictionary to store evaluation results
results = {}

# Iterate over each model with cross-validation
for model_name, model in models.items():
    # Cross-validate the model
    cv_accuracy = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy').mean()
    cv_f1 = cross_val_score(model, X_train, y_train, cv=kf, scoring='f1').mean()
    cv_auc = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc').mean()
    cv_mae = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error').mean() * -1  # Convert negative MAE to positive
    
    # Train the model on the full training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_test_pred = model.predict(X_test)
    
    # For AUC-ROC, we need the probability scores for the positive class
    y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Evaluate the model on the test set
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_auc = roc_auc_score(y_test, y_test_prob) if y_test_prob is not None else None
    
    # Check if any metric is 1.0, and skip such models
    if test_accuracy == 1.0 or test_auc == 1.0 or test_f1 == 1.0 or test_mae == 0.0:
        print(f"{model_name} is overfitting with a metric of 1.0, skipping this model.")
        continue
    
    # Store the evaluation results
    results[model_name] = {
        "CV Accuracy": cv_accuracy,
        "CV AUC": cv_auc,
        "CV MAE": cv_mae,
        "CV F1": cv_f1,
        "Test Accuracy": test_accuracy,
        "Test AUC": test_auc,
        "Test MAE": test_mae,
        "Test F1": test_f1
    }

# Display the results
print("\nModel Evaluation Results:")
for model_name, metrics in results.items():
    print(f"\n{model_name} Evaluation:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

# Filter out models with any 1.0 metric, if any are left
filtered_results = {model: metrics for model, metrics in results.items() 
                    if all(metric != 1.0 for metric in metrics.values())}

# If no models remain after filtering, display a message
if not filtered_results:
    print("\nNo models remain after filtering out overfitting models with metrics of 1.0.")
else:
    # Select the best model based on the balance of metrics (accuracy, AUC, F1, MAE)
    best_model = max(filtered_results, key=lambda x: (
        filtered_results[x]["Test Accuracy"] + 
        filtered_results[x]["Test AUC"] + 
        filtered_results[x]["Test F1"] - 
        filtered_results[x]["Test MAE"]
    ))  # Higher Accuracy, AUC, F1 and lower MAE are preferred

    print(f"\nBest Model: {best_model}")
    print(f"Best Model Metrics: {filtered_results[best_model]}")

    # Access the trained model object
    best_model_name = best_model
    best_model = models[best_model_name]  # This is the trained best model


In [ ]:
from lime.lime_tabular import LimeTabularExplainer
import numpy as np
import pandas as pd
import warnings

# Suppress specific warnings from LIME (optional)
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the LimeTabularExplainer
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['Not Approved', 'Approved'],
    mode='classification'
)
# Initialize an empty list to store explanations
explanations = []


# Suppress the specific warnings from the deprecated usage in lime
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the LimeTabularExplainer
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['Not Approved', 'Approved'],  # Adjust according to your dataset
    mode='classification'
)

# Pick a single instance from the test set to explain for testing
i = 0  # Index of the instance to explain

exp = lime_explainer.explain_instance(X_test.iloc[i], best_model.predict_proba)

# Display the explanation for the instance
exp.show_in_notebook(show_table=True)


In [ ]:
import re

# Suppress specific warnings from LIME (optional)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to extract the feature name from the LIME explanation
def extract_feature_name(feature_str):
    # Split on '<', '>', '=', '!=', etc., and take the first part
    split_symbols = ['<=', '>=', '<', '>', '!=', '=']
    for symbol in split_symbols:
        if symbol in feature_str:
            feature_name = feature_str.split(symbol)[0].strip()
            break
    else:
        feature_name = feature_str.strip()
    # Remove any numeric values
    feature_name = re.sub(r'[0-9]+', '', feature_name)
    # Remove any non-alphanumeric characters except spaces
    feature_name = re.sub(r'[^A-Za-z0-9 ]+', '', feature_name)
    # Replace multiple spaces with a single space
    feature_name = re.sub(r'\s+', ' ', feature_name)
    return feature_name.strip()

# Initialize the LimeTabularExplainer for the entire feature dataset `X`
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X),
    feature_names=X.columns,
    class_names=['Not Approved', 'Approved'],  # Adjust according to your dataset
    mode='classification'
)

# Initialize an empty list to store explanations
explanations = []

# Loop over the instances in the entire feature dataset `X`
for idx, instance in X.iterrows():
    # Get the model's prediction
    prediction = best_model.predict(instance.values.reshape(1, -1))[0]
    # Get the actual label from `y`
    actual_label = y.loc[idx]
    # Generate LIME explanation
    exp = lime_explainer.explain_instance(
        data_row=instance,
        predict_fn=best_model.predict_proba,
        num_features=len(X.columns)  # Consider all features
    )
    # Get the list of (feature, contribution) pairs
    explanation_list = exp.as_list()

    # Determine the decision (Approved or Denied)
    decision = 'Approved' if prediction == 1 else 'Denied'

    # Collect features contributing to the predicted class
    features_contributing = []
    for feature, weight in explanation_list:
        # For the predicted class, collect all features regardless of weight
        if (prediction == 1 and weight > 0) or (prediction == 0 and weight < 0):
            feature_name = extract_feature_name(feature)
            if feature_name and feature_name not in features_contributing:
                features_contributing.append(feature_name)

    # Construct the explanation text
    if features_contributing:
        explanation_text = f"This application was {decision.lower()} due to " + ", ".join(features_contributing) + "."
    else:
        explanation_text = f"This application was {decision.lower()}."

    # Append the explanation and related information
    explanations.append({
        'Index': idx,
        'Prediction': decision,
        'Actual': 'Approved' if actual_label == 1 else 'Denied',
        'Explanation': explanation_text
    })


In [ ]:
# Convert the explanations to a DataFrame
explanations_df = pd.DataFrame(explanations)

# Output the first explanation
first_explanation = explanations_df.iloc[0]

# Print the details
print("Index:", first_explanation['Index'])
print("Prediction:", first_explanation['Prediction'])
print("Actual:", first_explanation['Actual'])
print("Explanation:", first_explanation['Explanation'])


In [ ]:
explanations_df

In [ ]:
#reimport data due to earlier augmentation

df = pd.read_csv("../dataset/credit-card-approval/credit_card_approval.csv")
df

In [ ]:
# Add the 'Explanation' column (Reason) from explanations_df to the original df
df['Reason'] = explanations_df['Explanation']

# Save the updated dataframe with the new column as a CSV file
df.to_csv('../dataset/target-augmented.csv', index=False)

In [ ]:
df